In [ ]:
%matplotlib inline
import sys
sys.path.append('../../')
from ef.config.visualizer import Visualizer3d
from ef.config.efconf import EfConf
from ef.config.components import *

Dynamics of a single particle in free space is the simplest kind of simulation. It allows to check basic functionality of the program, and, besides, it is a good way to become familiar with how the program works.

In absence of external forces, a body moves in straight line with constant velocity

\begin{align}
& \textbf{r}(t) = \textbf{r}_0 + \textbf{v}_0 t
\\
& \textbf{v}(t) = \textbf{v}_0
\end{align}

To get some sense of scales (see below), suppose the particle is an electron that has just passed an 1 keV accelerating potential difference. It's mass `m` and charge `q` are `q = 4.8e-10 [cgs], m = 9.1e-28 [g]`. Since it's energy is nonrelativistic, it's possible to calculate it's speed simply as $v = \sqrt{ 2 E / m } = 1.808e+09 ~ [cm/s]$. To cover a `10 [cm]` distance with such speed, it will take the electron `t = 5.530e-09 [s] ~ 6 [ns]`.

In [ ]:
from math import *

m = 9.8e-28
q = 4.8e-10
print( "q = {:.3e} [cgs]".format( q ) )
print( "m = {:.3e} [g]".format( m ) )

ev_to_cgs = 1.60218e-12
E = 1000 * ev_to_cgs
v = sqrt( 2 * E / m )
z = 10
t = z / v
print( "E = {:.3e} [eV] = {:.3e} [erg]".format( E / ev_to_cgs, E ) )
print( "v = {:.3e} [cm/s]; p = {:.3e} [g * cm/s]".format( v, v * m ) )
print( "z = {:.3e} [cm]".format( z ) )
print( "t = {:.3e} [s]".format( t ) )

To perform a simulation, it's necessary to prepare a config file. 
First, it's necessary to set a total time of the simulation and a time step.
Let's use the estimates above as guiding values and set the total time to `6e-9 [s]`.
We use `1000` time steps, so that `time_step_size = 6.0e-12 [s]`. 
Besides, it's necessary to decide at which time steps the state of the whole simulation should be saved to the disk.
To save each step, `time_save_step` is set equal to `time_step_size`: `time_save_step = 6.0e-12`.

In [ ]:
#tmp_conf.visualize_all( v )

single_particle_in_free_space = EfConf()
vis = Visualizer3d()

single_particle_in_free_space.time_grid = TimeGrid(
    total = 6.0e-9,
    step = 6.0e-12,
    save_step = 6.0e-12
)

print( single_particle_in_free_space.export_to_string() )

Next we need to define size of a computational volume.
Domain size is set to 15 [cm] along the z-axis and 5 [cm] along the x- and y-axes. 
The parameters for PIC-mesh are also defined in this section.
Since we are going to use noninteracting particle model instead of PIC, they are not used and can be set arbitrary. 

In [ ]:
single_particle_in_free_space.spatial_mesh = SpatialMesh(
    size = ( 5.0, 5.0, 15.0 ),
    step = ( 0.5, 0.5, 1.5 )
)

print( single_particle_in_free_space.export_to_string() )

Next, a particle source. 
We need a single particle at startup ( `initial_number_of_particles = 1` ), approximately 1 [mm] away from
coordinate axis origin at the bottom-left-near corner of the domain (`box_x_left = 0.10` etc).
Charge and mass are set to those of the electron.
Momentum corresponding to 1 keV energy is `1.772e-18 [g * cm / s]`.
Such value is set for momentum along the z-axis. 
Momenta along the x- and y-axes could be set to zero; instead they are chosen 3 times smaller than the z-axis momentum
(domain size along the x and y is 3 times smaller than along the z). 
To prevent any variation in momentum, the `temperature` is set to 0.
Notice a source is given a descriptive: `emit_single_particle` in this case.

In [ ]:
# todo: add method to add single source

single_particle_in_free_space.sources = [ 
    ParticleSource(
        name = "emit_single_particle",
        initial_particles = 1,
        particles_to_generate_each_step = 0,
        #shape = Box( origin = (0.1, 0.1, 0.1), size = ( 0.01, 0.01, 0.01 ) ),
        shape = Box( origin = (0.1, 0.1, 0.1), size = ( -0.01, 0.01, 0.01 ) ), # hack left > right error
        momentum = ( 0.6e-18, 0.6e-18, 1.77e-18 ),
        temperature = 0.0,
        charge = 4.8e-10,
        mass = 9.8e-28
    )
]

print( single_particle_in_free_space.export_to_string() )

Next section is boundary conditions. 
In a free space, potentials on each boundary should be equal. 
It is possible to simply set them to zero.

In [ ]:
single_particle_in_free_space.boundary_conditions = BoundaryConditions( potential = 0 )

print( single_particle_in_free_space.export_to_string() )

Particle interaction model allows to choose between noninteracting particles and particles interacting by
particle-in-cell method. Since there is only one particle, there is no need to use PIC, and noninteracting mode is enough. Moreover, PIC will lead to wrong results (because particle creates electric field that acts back on the particle). 

In [ ]:
single_particle_in_free_space.particle_interaction_model = ParticleInteractionModel(
    model = "noninteracting"
)

print( single_particle_in_free_space.export_to_string() )

The last step is to specify pattern for output file names. 
They will be of the form `single_particle_free_space_0001000.h5`, where `0001000` is a time step number.

In [ ]:
single_particle_in_free_space.output_file = OutputFile(
    prefix = "single_particle_free_space_",
    suffix = ".h5"
)

print( single_particle_in_free_space.export_to_string() )

To start the simulation, the config should be provided as an argument to the `ef.out`.

In [ ]:
from ef.util.runner import EfRunner


runner = EfRunner( conf = single_particle_in_free_space, ef_command="python3 ../../main.py" )
runner.run()

After the simulation finishes, a bunch of `*.h5` files will emerge in the directory.
To open them and glance over the results, an [Hdfview](https://support.hdfgroup.org/products/java/hdfview/) is a convenient tool.
In this simulation, two places are of interest: `current time` and particle position
and momentum at this time moment:

<p align="center">
<img src="https://github.com/epicf/ef/blob/dev/doc/figs/single_particle_in_free_space/hdfview_results.png" alt="hdfview screenshot" width="800"/>
</p>


Each output `h5`-file corresponds to a different time step. 
If we were able to extract time, position and momentum of the particle from each file, we could have obtained numerical trajectory, which could be compared to the analytical one. 

Data extraction and plotting are accomplished by the accompanying [`plot.py`](https://github.com/epicf/ef/blob/master/examples/single_particle_in_free_space/plot.py) python script.
It's `main` function does exactly what is described above: extracts numerical trajectory from the output
files, evaluates analytical trajectory at the extracted time points and plots both numerical and analytical
trajectories on the same axes.  

In [ ]:
def main():
    num = extract_num_trajectory_from_out_files()
    an = eval_an_trajectory_at_num_time_points( num )
    plot_trajectories( num , an )

It uses [h5py](http://www.h5py.org/) library to read from h5 files, [numpy](http://www.numpy.org/) to simplify working with arrays and [matplotlib](https://matplotlib.org/) to perform plotting of the trajectory. 

In [ ]:
import os, glob
import h5py
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D 

Let's examine the function to obtain the numerical trajectory:

In [ ]:
def extract_num_trajectory_from_out_files():
    out_files = find_necessary_out_files()                           # (*1)

    num_trajectory = []
    for f in out_files:
        num_trajectory.append( extract_time_pos_mom( f ) )           # (*2)  

    num_trajectory = remove_empty_and_sort_by_time( num_trajectory ) # (*3)
    num_trajectory = np.array( num_trajectory,                       # (*4)
                               dtype=[('t','float'),
                                      ('x','float'), ('y','float'), ('z','float'),
                                      ('px','float'), ('py','float'),('pz','float')])
    return( num_trajectory )

(\*1): To extract data, first we find all *.h5 files in the current directory.  
(\*2): Then the script iterates over them and extracts a relevant data from each one.  
(\*3): In the end of this process, `num_trajectory` array contains positions and velocities of particle at different
time steps. In is convenient to sort it over time values.  
(\*4): After the previous step the numerical trajectory is in a format of a list of tuples:
`[ (t, x, y, z, px, py, pz), ..... ]` . It is convenient to convert it into one of the `np.array` types to simplify further manipulations with it.  


For analytical trajectory we need the initial position and momentum.
They can be extracted from the first h5-file:

In [ ]:
def get_mass_and_initial_pos_and_mom():
    initial_out_file = "single_particle_free_space_0000000.h5"
    h5 = h5py.File( initial_out_file, mode="r")
    m = h5["/Particle_sources/emit_single_particle"].attrs["mass"][0]
    x0 = h5["/Particle_sources/emit_single_particle/position_x"][0]
    y0 = h5["/Particle_sources/emit_single_particle/position_y"][0]
    z0 = h5["/Particle_sources/emit_single_particle/position_z"][0]
    px0 = h5["/Particle_sources/emit_single_particle/momentum_x"][0]
    py0 = h5["/Particle_sources/emit_single_particle/momentum_y"][0]
    pz0 = h5["/Particle_sources/emit_single_particle/momentum_z"][0]
    h5.close()
    return( m, x0, y0, z0, px0, py0, pz0 )

With this function, the analytical trajectory can be computed at the same 
time points as the numerical one:

In [ ]:
def eval_an_trajectory_at_num_time_points( num_trajectory ):
    global particle_mass
    particle_mass, x0, y0, z0, px0, py0, pz0 =  get_mass_and_initial_pos_and_mom()
    #
    an_trajectory = np.empty_like( num_trajectory )
    for i, t in enumerate( num_trajectory['t'] ):
        x, y, z = coords( particle_mass, t, x0, y0, z0, px0, py0, pz0 )
        px, py, pz = momenta( t, px0, py0, pz0 )
        an_trajectory[i] = ( t, x, y ,z, px, py, pz )
    #
    return( an_trajectory )

Finally, both numerical and analytical trajectories are plotted in 3d and side views. 
Kinetic energies are also compared.

In [ ]:
def plot_trajectories( num , an ):
    plot_3d( num, an )
    plot_2d( num, an )
    plot_kin_en( num , an )

This results in the following figures:
    
    _insert figs_

It can be seen, that energy is conserved during the simulation:

_insert fig_